# Caso de uso

### Alocação de recursos: Resistência do concreto

Escolhi fazer o tech challenge focado no estudo de resistência (Feature Compression Know - FCK) do concreto por dois motivos:

1. Minha primeira formação é em engenharia civil e sei que esse é um dos cálculos mais realizados na área a ponto de ser feito e refeito várias vezes mesmo utilizando sistemas para que seja garantido a segurança quando a obra estiver pronta e não haja nenhuma ruptura/trinca no concreto;

2. Gosto do assunto mesmo tendo migrado para a área de tecnologia.

### Objetivo

O objetivo aqui será encontrar a composição de concreto que fornece a maior resistência (FCK) em MPa dentro da população fornecida.

<span style="color:red">Importante ressaltar que todo concreto deve ser estruturado e estudado conforme a necessidade da obra, sendo assim, nenhum resultado que for obtido aqui poderá e nem deverá ser utilizado em nenhuma obra de construção civil sem prévio estudo do fim da utilização do concreto.</span>

# Importando os dados

In [1]:
import pandas as pd
import random
import itertools
import math

dataset = pd.read_csv("./base-desafio/concrete.csv")

dataset.head()


,cimento,escória,cinza,água,superplastificante,brita,areia,idade,resistência
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28,29.89
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22
3,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
4,154.8,183.4,0.0,193.3,9.1,1047.4,696.7,28,18.29


# Exploração dos dados

In [2]:
dataset.shape

(1030, 9)

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cimento             1030 non-null   float64
 1   escória             1030 non-null   float64
 2   cinza               1030 non-null   float64
 3   água                1030 non-null   float64
 4   superplastificante  1030 non-null   float64
 5   brita               1030 non-null   float64
 6   areia               1030 non-null   float64
 7   idade               1030 non-null   int64  
 8   resistência         1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [4]:
dataset.isnull().sum()

cimento               0
escória               0
cinza                 0
água                  0
superplastificante    0
brita                 0
areia                 0
idade                 0
resistência           0
dtype: int64

In [5]:
dataset.duplicated().sum()

25

In [6]:
dataset.drop_duplicates(inplace=True)

In [7]:
dataset.shape

(1005, 9)

Não temos dados nulos e o que tínhamos de dados duplicados já foram removidos, então podemos seguir com o processo de criação do algoritmo genético

# Parâmetros

In [8]:
# Resistência à compressão do concreto target
fck_target = 100

# Parâmetros de entrada
tamanho_populacao = 10
numero_geracoes = 10
probabilidade_mutacao = 0.5
composicao_concretos = dataset.values

# População inicial

In [42]:
def euclidean_distance(concreto1, concreto2):
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(concreto1, concreto2)))

def nearest_neighbor(composicao_concretos, inicio):
    concretos_nao_avaliados = set(map(tuple, composicao_concretos))
    concretos_nao_avaliados.remove(inicio)
    atual = inicio
    caminho = [atual]
    
    while concretos_nao_avaliados:
        # Filtra os concretos não visitados para garantir que nenhum material seja zero ou negativo e que a resistência à compressão seja menor ou igual ao target
        concretos_validos = [concreto for concreto in concretos_nao_avaliados if concreto[0] > 0 and concreto[3] > 0 and concreto[5] >0 and concreto[6] > 0 and concreto[7] > 0 and concreto[-1] <= fck_target]
        
        if not concretos_validos:
            break
        proximo_concreto = min(concretos_validos, key=lambda x: euclidean_distance(atual, x))
        concretos_nao_avaliados.remove(proximo_concreto)
        caminho.append(proximo_concreto)
        atual = proximo_concreto
    
    return caminho

def gera_populacao(composicao_concretos, tamanho_populacao):
    populacao = []
    lista_concretos = list(map(tuple, composicao_concretos))
    
    for _ in range(tamanho_populacao):
        inicio = random.choice(lista_concretos)
        individuo = nearest_neighbor(lista_concretos, inicio)
        populacao.append(individuo)
    
    return sum(populacao, [])

# Função fitness

In [30]:
def calcula_resistencia(concreto):
    return 1 / (1 + abs(fck_target - concreto[-1]))

# Sort população pelo fitness

In [28]:
def sort_populacao(populacao):
  return sorted(populacao,key=calcula_resistencia,reverse=True)

# Crossover

In [12]:
def crossover(origem1, origem2):
    crossover = random.randint(0, len(origem1))
    filho1 = list(origem1[:crossover]) + [material for material in origem2 if material not in origem1[:crossover]]
    filho2 = list(origem2[:crossover]) + [material for material in origem1 if material not in origem2[:crossover]]
    return filho1, filho2

# Mutação

In [13]:
def mutacao(filho, probabilidade_mutacao):
    if random.random() < probabilidade_mutacao:
        gene1, gene2 = random.sample(range(len(filho)), 2)
        filho[gene1], filho[gene2] = filho[gene2], filho[gene1]
    return filho

# Algoritmo Genético

In [43]:
melhores_fitness_valores = []
melhores_concretos = []
contador_geracao = itertools.count(start=1)
geracao = 0
populacao = gera_populacao(composicao_concretos, tamanho_populacao)
melhor_concreto = populacao[0]

print(f"População inicial: {populacao}")
    
while geracao < numero_geracoes:
    geracao = next(contador_geracao)

    populacao = sort_populacao(populacao)
    print(f"População ordenada: {populacao}")
    melhor_fitness = calcula_resistencia(populacao[0])
    melhor_concreto = populacao[0]
    melhores_fitness_valores.append(melhor_fitness)
    melhores_concretos.append(melhor_concreto)


    # Elitismo
    nova_populacao = [populacao[0]]

    while len(nova_populacao) < tamanho_populacao:
        origem1, origem2 = random.choices(populacao[:10], k=2)
        filho1, filho2 = crossover(origem1, origem2)
        filho1 = mutacao(filho1, probabilidade_mutacao)
        filho2 = mutacao(filho2, probabilidade_mutacao)
        nova_populacao.extend([filho1, filho2])
        
    populacao = nova_populacao[:tamanho_populacao]
    print(f"==========================Geração {geracao}=================================")
    print(f"População nova: {populacao}")
    melhor_concreto = populacao[0]
    print(f"Melhor concreto da geração: {melhor_concreto} com fitness {melhor_fitness*100}")
    print("===========================================================================")
    if melhor_concreto[-1] == fck_target:
        break

População inicial: [(272.8, 181.9, 0.0, 185.7, 0.0, 1012.4, 714.3, 7.0, 19.77), (272.8, 181.9, 0.0, 185.7, 0.0, 1012.4, 714.3, 28.0, 31.38), (255.5, 170.3, 0.0, 185.7, 0.0, 1026.6, 724.3, 28.0, 32.05), (255.5, 170.3, 0.0, 185.7, 0.0, 1026.6, 724.3, 7.0, 17.24), (238.2, 158.8, 0.0, 185.7, 0.0, 1040.6, 734.3, 7.0, 15.69), (238.2, 158.8, 0.0, 185.7, 0.0, 1040.6, 734.3, 28.0, 26.91), (220.8, 147.2, 0.0, 185.7, 0.0, 1055.0, 744.3, 28.0, 25.75), (220.8, 147.2, 0.0, 185.7, 0.0, 1055.0, 744.3, 7.0, 13.09), (203.5, 135.7, 0.0, 185.7, 0.0, 1076.2, 759.3, 7.0, 11.96), (186.2, 124.1, 0.0, 185.7, 0.0, 1083.4, 764.3, 7.0, 8.0), (186.2, 124.1, 0.0, 185.7, 0.0, 1083.4, 764.3, 28.0, 17.6), (203.5, 135.7, 0.0, 185.7, 0.0, 1076.2, 759.3, 28.0, 22.63), (213.8, 98.1, 24.5, 181.7, 6.7, 1066.0, 785.5, 14.0, 17.84), (213.7, 98.1, 24.5, 181.7, 6.9, 1065.8, 785.4, 3.0, 18.0), (213.8, 98.1, 24.5, 181.7, 6.7, 1066.0, 785.5, 3.0, 13.18), (213.7, 98.1, 24.5, 181.7, 6.9, 1065.8, 785.4, 14.0, 30.39), (213.8, 98.1, 24

# Conclusão

```markdown

O estudo realizado teve como objetivo encontrar a composição de concreto que fornece a maior resistência à compressão (FCK) em MPa dentro da população fornecida. Utilizando um algoritmo genético, foi possível simular e otimizar diferentes composições de concreto, levando em consideração parâmetros como cimento, escória, cinza, água, superplastificante, brita, areia e idade.

Os resultados obtidos demonstraram que o algoritmo genético é uma ferramenta eficaz para a otimização de composições de concreto, permitindo a identificação de misturas que atendem aos critérios de resistência desejados. Através da seleção, crossover e mutação, o algoritmo foi capaz de evoluir a população inicial de concretos, melhorando gradativamente a resistência à compressão das composições.

É importante ressaltar que, embora os resultados sejam promissores, eles não devem ser utilizados diretamente em aplicações reais sem uma análise e validação adicionais. Cada obra de construção civil possui requisitos específicos que devem ser considerados, e a segurança estrutural deve ser garantida por meio de estudos detalhados e ensaios laboratoriais.

Em suma, o uso de algoritmos genéticos para a otimização de composições de concreto mostrou-se uma abordagem viável e eficiente, contribuindo para avanços na engenharia civil e na busca por materiais de construção mais resistentes e duráveis.
```